In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, GlobalAveragePooling1D,GaussianNoise
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
train = pd.read_csv('/content/Drive/My Drive/Colab Notebooks/cod_xrd_42k.csv')
test = pd.read_csv('/content/Drive/My Drive/Colab Notebooks/test.csv')

In [ ]:
train

,formula,spacegroup_number,cod_id,bravais_lattice,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,...,y140,y141,y142,y143,y144,y145,y146,y147,y148,y149,y150,y151,y152,y153,y154,y155,y156,y157,y158,y159,y160,y161,y162,y163,y164,y165,y166,y167,y168,y169,y170,y171,y172,y173,y174,y175,y176,y177,y178,y179
0,H26C25N4ClO3F,2,2227539,aP,4.909847e-137,2.993258e-111,3.522734e-88,8.003409e-68,3.510182e-50,2.971966e-35,4.857552e-23,1.532677e-13,9.335620e-07,1.097729e-02,2.491764e-01,1.091887e-02,1.296074e-05,2.825521e-02,1.280369e-01,1.120043e-03,6.600268e-04,8.624877e-02,2.217312e-02,1.194057e-05,5.475616e-05,1.453245e-03,7.445677e-05,1.009178e-08,3.252937e-04,1.180617e-01,2.200547e-01,1.347891e-03,1.797099e-02,9.547305e-02,1.331627e-02,1.252746e-01,5.068734e-01,1.075955e-01,1.926852e-02,5.503531e-01,...,0.029190,0.030961,0.032077,0.022185,0.027930,0.028955,0.012661,2.310287e-02,1.645867e-02,1.911054e-02,2.108801e-02,1.928122e-02,0.022688,0.015058,0.014596,0.019487,0.017810,1.490268e-02,0.015490,0.031300,0.025390,0.020524,0.025057,0.015366,0.016199,0.026934,0.024970,1.882013e-02,2.076732e-02,0.008412,0.023088,0.034702,2.624012e-02,0.018113,0.013302,0.019432,0.030183,1.792260e-02,2.285690e-02,6.056704e-03
1,CoP3H36C23,2,4085709,aP,3.391626e-189,1.235121e-158,8.683022e-131,1.178397e-105,3.087251e-83,1.561391e-63,1.524442e-46,2.873224e-32,1.045412e-20,7.342861e-12,9.956403e-06,2.606148e-02,1.316907e-01,1.284609e-03,2.419056e-08,8.794368e-16,3.124754e-11,3.856386e-05,9.222466e-02,4.795855e-01,4.767336e-01,1.000000e+00,4.330130e-01,9.362602e-01,2.370594e-01,6.057618e-01,4.829572e-02,4.564380e-05,2.264387e-02,3.322019e-01,6.440661e-01,5.352813e-01,2.727797e-02,1.312622e-01,1.975869e-01,1.450226e-01,...,0.021089,0.035128,0.021237,0.018717,0.030170,0.021675,0.022245,1.013127e-02,3.006732e-02,2.783368e-02,3.130481e-02,2.181813e-02,0.028440,0.030898,0.013406,0.013907,0.017776,1.729460e-02,0.021639,0.016083,0.025751,0.017797,0.018680,0.008578,0.020196,0.029689,0.045545,2.351549e-02,2.116400e-02,0.029617,0.013113,0.013948,1.540255e-02,0.013787,0.022637,0.021859,0.020878,2.819485e-02,2.273572e-02,1.407936e-02
2,H34C34S2(N2O)3,2,2210818,aP,6.144995e-218,5.954486e-185,1.113850e-154,4.022243e-127,2.803945e-102,3.773377e-80,9.802807e-61,4.916207e-44,4.759588e-30,8.895452e-19,3.209417e-10,2.235341e-04,3.005529e-01,7.801132e-01,3.993161e-03,5.030111e-02,1.000000e+00,4.850407e-02,4.553853e-06,8.253745e-13,9.237974e-22,5.714583e-13,9.927731e-07,3.329496e-03,2.301766e-02,1.535956e-01,5.407014e-02,9.275807e-03,1.041866e-03,2.867258e-02,9.546877e-03,5.995847e-02,1.361531e-01,5.672377e-01,2.979968e-02,7.537629e-02,...,0.012713,0.010919,0.009939,0.005539,0.016311,0.015051,0.017017,7.652663e-03,9.325286e-03,1.252722e-02,5.989840e-03,8.790821e-03,0.014130,0.011555,0.013917,0.016375,0.014893,1.746344e-02,0.017212,0.008634,0.019216,0.022922,0.021809,0.016341,0.009269,0.018402,0.011803,1.654706e-02,1.122908e-02,0.007848,0.015220,0.012785,9.240285e-03,0.013426,0.016862,0.025352,0.009457,8.136539e-03,6.792385e-03,4.124101e-03
3,P2H44Ru2C56N14Cl4O9.25F12,2,4312692,aP,4.146075e-83,1.397867e-63,9.098168e-47,1.143146e-32,2.772737e-21,1.298301e-12,1.173549e-06,2.047797e-03,6.898139e-03,4.485766e-05,2.838280e-05,1.212533e-01,1.000000e+00,1.681565e-02,1.816843e-01,2.805747e-01,2.410864e-01,1.289372e-01,1.858789e-04,5.996919e-02,1.435967e-01,2.248196e-01,1.414138e-01,5.643617e-01,2.615184e-02,1.302231e-01,1.273463e-01,1.106189e-01,1.491101e-01,1.118869e-01,7.217123e-02,4.873515e-03,6.046018e-03,7.964048e-02,2.942634e-02,1.126407e-02,...,0.010671,0.009706,0.012130,0.010644,0.009180,0.010826,0.010808,1.419333e-02,1.707341e-02,1.254624e-02,7.769792e-03,1.408236e-02,0.012897,0.012506,0.014709,0.013707,0.009151,1.098745e-02,0.010509,0.008747,0.006850,0.012347,0.014479,0.007145,0.009126,0.012043,0.011211,1.076932e-02,1.129822e-02,0.007996,0.006726,0.009492,1.115840e-02,0.009486,0.009029,0.013752,0.0

In [ ]:
hp = train.loc[train['bravais_lattice'] =="hP"]
ms = train.loc[train['bravais_lattice'] =="mS"]
ap = train.loc[train['bravais_lattice'] =="aP"]
hr = train.loc[train['bravais_lattice'] =="hR"]
op = train.loc[train['bravais_lattice'] =="oP"]
mp = train.loc[train['bravais_lattice'] =="mP"]
tp = train.loc[train['bravais_lattice'] =="tP"]
cF = train.loc[train['bravais_lattice'] =="cF"]
tI = train.loc[train['bravais_lattice'] =="tI"]
os = train.loc[train['bravais_lattice'] =="oS"]
cp = train.loc[train['bravais_lattice'] =="cP"]
ci = train.loc[train['bravais_lattice'] =="cI"]
oi = train.loc[train['bravais_lattice'] =="oI"]
of = train.loc[train['bravais_lattice'] =="oF"]

In [ ]:
hp_resampled = resample(hp,replace=False, n_samples=4000)
ms_resampled = resample(ms,replace=False, n_samples=4000)
ap_resampled = resample(ap,replace=False, n_samples=4000)
hr_resampled = resample(hr,replace=False, n_samples=4000)
op_resampled = resample(op,replace=False, n_samples=4000)
mp_resampled = resample(mp,replace=False, n_samples=4000)
tp_resampled = resample(tp,replace=True, n_samples=4000)
cF_resampled = resample(cF,replace=True, n_samples=4000)
tI_resampled = resample(tI,replace=True, n_samples=4000)
os_resampled = resample(os,replace=True, n_samples=4000)
cp_resampled = resample(cp,replace=True, n_samples=4000)
ci_resampled = resample(ci,replace=True, n_samples=4000)
oi_resampled = resample(oi,replace=True, n_samples=4000)
of_resampled = resample(of,replace=True, n_samples=4000)
resampled_data = pd.concat([hp_resampled,ms_resampled,ap_resampled,
                           hr_resampled, op_resampled,mp_resampled,tp_resampled,cF_resampled,
                           tI_resampled, os_resampled,cp_resampled,ci_resampled,oi_resampled,
                           of_resampled])

In [ ]:
labels = resampled_data.iloc[:,3]
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_transformed =  le.transform(labels)
dummy_y = np_utils.to_categorical(labels_transformed)
X = resampled_data.iloc[:,4:184]
X_arr= X.to_numpy()
X_updated = X_arr.reshape(56000,180,1)

In [ ]:
trian_orig = train.iloc[:,4:184]
trian_orig_arr = trian_orig.to_numpy()
train_orig_updated = trian_orig_arr.reshape(41814,180,1)
labels_orig = train.iloc[:,3]

In [ ]:
labels.value_counts()

hR    4000
oP    4000
cP    4000
tI    4000
oI    4000
mS    4000
aP    4000
tP    4000
mP    4000
cI    4000
hP    4000
oS    4000
oF    4000
cF    4000
Name: bravais_lattice, dtype: int64

In [ ]:
def baseline_model():
	# create model
  model = Sequential()
  #model.add(Conv1D(100, 3, activation='relu', input_shape=[180,1]))
  model.add(Conv1D(32, 3, input_shape=[180,1]))
  model.add(Conv1D(32, 3, activation='relu'))
  model.add(MaxPooling1D(3))
  model.add(Dropout(0.2))
  model.add(Conv1D(64, 3, activation='relu'))
  model.add(Conv1D(64, 3, activation='relu'))
  model.add(MaxPooling1D(3))
  model.add(Dropout(0.2))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(MaxPooling1D(3)) 
  model.add(Dropout(0.3)) 
  model.add(Flatten())
  #model.add(Dense(90, activation='tanh'))
  #model.add(GaussianNoise(0.1))
  #model.add(Dense(50, activation='tanh'))
  #model.add(GaussianNoise(0.1))
  model.add(Dense(14, activation='softmax'))
	# Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

#baseline_model = baseline_model()
#history_CNN_classify = baseline_model.fit(X_train_updated, y_train, validation_data=(X_test_updated, y_test), epochs=60, batch_size=40, verbose=1)

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=60, batch_size=40,verbose=0)
estimator.fit(X_updated, dummy_y)
predictions = estimator.predict(train_orig_updated)
print(predictions)
print(le.inverse_transform(predictions))

[10  6  6 ... 10 13 13]
['oP' 'mP' 'mP' ... 'oP' 'tP' 'tP']


In [ ]:
#kfold = KFold(n_splits=5, shuffle=True)
#results = cross_val_score(estimator, X_test_updated, y_test, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
print(np.sum(le.inverse_transform(predictions)==labels_orig)/len(labels_orig))

0.672382455636868


In [ ]:
#ids = test.iloc[:,0]
#test_labels = test.iloc[:,1:181].to_numpy()
#test_updated = test_labels.reshape(4384,180,1)
#test_pred = estimator.predict(test_updated)
#y_pred_transformed = le.inverse_transform(test_pred)
#ans = np.column_stack((ids, y_pred_transformed))
#dataset = pd.DataFrame({'id': ans[:, 0], 'bravais_lattice': ans[:, 1]})
#dataset.to_csv (r'/content/Drive/My Drive/Colab Notebooks/pred_CNN_resample.csv', index = False, header=True)
#view = pd.read_csv('/content/Drive/My Drive/Colab Notebooks/pred_CNN_resample.csv')
#view